In [101]:

import pandas as pd
import warnings
warnings.filterwarnings("ignore")


In [102]:
data = pd.read_csv('../data/spare_cleaned.csv')
data.head()

,invoice_date,vehicle_model,invoice_line
0,2017-05-30,BAJAJ AVENGER STREET 220,ENGINE OIL
1,2017-06-02,BAJAJ PULSAR NS 200,ENGINE OIL
2,2017-06-02,BAJAJ PULSAR NS 200,POLISH
3,2017-06-02,BAJAJ PULSAR NS 200,CONSUMABLES
4,2017-06-02,BAJAJ PULSAR NS 200,COOLANT OIL


 # ABC Analysis
 ABC analysis is an approach for classifying inventory items based on the items consumption values. Consumption value is the total value of an item consumed over a specified time period, for example a year. The approach is based on the Pareto principle to help manage what matters and is applied in this context:

 ## A B C class separation
 * **A Class** is top 50%
 * **B Class** is top 50-80%
 * **C Class** is the rest

In [103]:
data.shape


(28448, 3)

In [104]:

def get_invoice_line_by_consumption(data: pd.DataFrame, start_consumption: float, end_consumption: float) -> (pd.DataFrame, float, float):
    """ start_consumption: float between 0 and 1 which denotes the top percentage of data as a ration between 0 and 1
    end_consumption: float between 0 and 1 which denotes the top percentage of data as a ration between 0 and 1
    We want to return actual start consumption and end consumption based on data because we want to 
    either select the spare part or drop it completely
    
    returns: a tuple with (
    data frame with the top consumption data,
    actual start consumption used,
    actual end consumption used
    )

    """

    length = data.shape[0]
    df = data[['invoice_date', 'invoice_line']].groupby('invoice_line').count().sort_values('invoice_date', ascending=False)
    
    df['cum_sum'] = df['invoice_date'].cumsum()
    df['cum_sum_perc'] = df['cum_sum'] / length
    df1 = df.loc[ (df['cum_sum_perc'] > start_consumption) & (df['cum_sum_perc'] <= end_consumption)]
    
    actual_start = (df1['cum_sum'].iloc[0] - df1['invoice_date'].iloc[0]) / length
    actual_end = df1['cum_sum_perc'].iloc[-1]
    
    return df1, actual_start, actual_end
    
    
    
print(get_invoice_line_by_consumption(data, 0.0, .5))
get_invoice_line_by_consumption(data, .5, .8)

(                   invoice_date  cum_sum  cum_sum_perc
invoice_line                                          
ENGINE OIL                 3802     3802      0.133647
CHAIN LUBRICATION          3441     7243      0.254605
GENERAL SERVICE            2142     9385      0.329900
AIR FILTER                 1715    11100      0.390186
3M OIL                     1628    12728      0.447413, 0.0, 0.4474128233970754)


(                     invoice_date  cum_sum  cum_sum_perc
 invoice_line                                            
 CONSUMABLES                  1595    14323      0.503480
 POLISH                       1245    15568      0.547244
 DISC OIL                      991    16559      0.582080
 BRAKE SHOE                    965    17524      0.616001
 OIL FILTER                    821    18345      0.644861
 DISC PAD                      575    18920      0.665073
 WHEEL RUBBER                  521    19441      0.683387
 AIR FILTER CHECKUP            467    19908      0.699803
 SPARK PLUG                    421    20329      0.714602
 CHAIN SPROCKET                396    20725      0.728522
 SPROCKET RUBBER               347    21072      0.740720
 SPROCKET BEARING              314    21386      0.751758
 CHAIN OVERHAUL                284    21670      0.761741
 CLUTCH CABLE                  263    21933      0.770986
 CLUTCH ASSEMBLY               212    22145      0.778438
 CLUTCH COVER 

In [105]:
def get_A_B_C(data: pd.DataFrame, a_end: float, b_end: float) -> (pd.DataFrame, pd.DataFrame, pd.DataFrame):
    """
    data : dataframe
    a_end : Where A class ends
    b_end : where B class ends
    returns : return a tuple which contains 3 data frames A B C
    
    """
    
    a_category_df_counts, a_category_start, a_category_end = get_invoice_line_by_consumption(data, 0.0, 0.5)
    b_category_df_counts, b_category_start, b_category_end = get_invoice_line_by_consumption(data, 0.5, 0.8)
    c_category_df_counts, c_category_start, c_category_end = get_invoice_line_by_consumption(data, 0.8, 1.0)
    
    a_category_df = data.loc[data['invoice_line'].isin(a_category_df_counts.index)]
    b_category_df = data.loc[data['invoice_line'].isin(b_category_df_counts.index)]
    c_category_df = data.loc[data['invoice_line'].isin(c_category_df_counts.index)]
    
    return a_category_df, b_category_df, c_category_df
    

In [106]:
s = 0
for f, df in zip(['A', 'B', 'C'], get_A_B_C(data,0.5,0.8)):
    filename = f'../data/{f}.csv'
    df.to_csv(filename, index=False)
    s += len(df)
    
assert len(data) == s